In [9]:
import numpy as np
import einops

In [25]:
import numpy as np
from numba import njit

def orthogonal_complement(x, normalize=False, threshold=1e-15):
    """Compute orthogonal complement of a matrix

    this works along axis zero, i.e. rank == column rank,
    or number of rows > column rank
    otherwise orthogonal complement is empty

    TODO possibly: use normalize='top' or 'bottom'

    """
    x = np.asarray(x)
    r, c = x.shape
    if r < c:
        import warnings
        warnings.warn('fewer rows than columns', UserWarning)

    # we assume svd is ordered by decreasing singular value, o.w. need sort
    s, v, d = np.linalg.svd(x)
    rank = (v > threshold).sum()

    oc = s[:, rank:]

    if normalize:
        k_oc = oc.shape[1]
        oc = oc.dot(np.linalg.inv(oc[:k_oc, :]))
    return oc



def pre_maxvol(matrix, rank):
    res = np.int64(np.zeros(rank))
    row_norms = np.linalg.norm(matrix, axis = 1) ** 2
    
    for k in range(rank):
        maxj = np.argmax(row_norms)
        res[k] = maxj
        oc = orthogonal_complement(matrix[res[:k+1]].T)
        row_norms = np.linalg.norm(matrix @ oc, axis = 1) ** 2
        row_norms[res[:k+1]] = -1
    return res


@njit
def search_start_columns(matrix, start_rows, threshold=1e-16):
    mat = matrix[start_rows]
    q, r = np.linalg.qr(mat)
    rank = len(start_rows)
    res = np.zeros(rank)
    cur_i = 0
    shift = 0
    for i in range(r.shape[1]):
        if np.sum(np.abs(r[:, i]) > threshold) > cur_i:
            res[cur_i] = i
            cur_i += 1
    return res


def maxvol_rows(matrix, start_rows, start_columns, eps=1e-3, max_iters=50):
    current_rows = start_rows
    current_columns = start_columns
    # начниаем поиск в строках
    inv_matrix = np.linalg.inv(matrix[current_rows, :][:, current_columns])
    
    swaps = 0
    
    for iter in range(max_iters):
        # домножаем на обратную
        search_matrix = inv_matrix @ matrix[current_rows, :]
        
        # ищем элемент по модулю больший единицы
        max_index = np.unravel_index(np.argmax(np.abs(search_matrix)), (len(current_rows), matrix.shape[1]))
    
        if np.abs(search_matrix[max_index[0], max_index[1]]) <= 1 + eps:
            # если попали сюда, значит, не нашли
            break
            
        swaps+= 1
        # обновляем столбец
        
        old_col = current_columns[max_index[0]]
        current_columns[max_index[0]] = max_index[1]
        
        # применяем формулу Шермана-Моррисона для обновления обратной матрицы
        u = matrix[current_rows, current_columns[max_index[0]]] - matrix[current_rows, old_col]
        v = np.zeros(len(current_rows))
        v[max_index[0]] = 1
        
        inv_matrix += inv_matrix - inv_matrix @ u[:, None] @ v[None, :] @ inv_matrix / \
                (1 + v[None, :] @ inv_matrix @ u[:, None])
        
    return (current_rows, current_columns, swaps)

def maxvol_columns(matrix, start_rows, start_columns, eps=1e-3, max_iters=50):
    current_rows = start_rows
    current_columns = start_columns
    # начниаем поиск в строках
    
    inv_matrix = np.linalg.inv(matrix[current_rows, :][:, current_columns])
    
    swaps = 0
    for iter in range(max_iters):
        # домножаем на обратную
        search_matrix = matrix[:, current_columns] @ inv_matrix
        
        # ищем элемент по модулю больший единицы
        max_index = np.unravel_index(np.argmax(np.abs(search_matrix)), (matrix.shape[0], len(current_columns)))
    
        if np.abs(search_matrix[max_index[0], max_index[1]]) <= 1 + eps:
            # если попали сюда, значит, не нашли
            break
        
        swaps += 1
        # обновляем столбец
        old_row = current_rows[max_index[1]]
        current_rows[max_index[1]] = max_index[0]
        
        # применяем формулу Шермана-Моррисона для обновления обратной матрицы
        u = np.zeros(len(current_columns))
        u[max_index[1]] = 1
        v = matrix[current_rows[max_index[1]], current_columns] - matrix[old_row, current_columns]
        
        inv_matrix += inv_matrix - inv_matrix @ u[:, None] @ v[None, :] @ inv_matrix / \
                (1 + v[None, :] @ inv_matrix @ u[:, None])
        
    return (current_rows, current_columns, swaps)


def maxvol(matrix, start_rows, start_columns, eps=1e-3, max_iters=50):
    current_rows = start_rows
    current_columns = start_columns
    swaps = 1
    while swaps: 
        current_rows, current_columns, swaps = maxvol_rows(matrix, current_rows, current_columns, eps, max_iters)
        if not swaps:
            break
        current_rows, current_columns, swaps = maxvol_columns(matrix, current_rows, current_columns, eps, max_iters)
    return matrix[current_rows, :], matrix[current_rows, current_columns], matrix[:, current_columns]


def search_max_volume_submatrix(matrix, rank, eps=1e-4, zero_threshold=1e-16, max_iters=50):
    matrix = np.float64(matrix)
    # ищем неплохое начальное приближение
    start_rows = pre_maxvol(matrix, rank)

    start_columns = np.int64(search_start_columns(matrix, start_rows, zero_threshold))

    start_rows = np.sort(start_rows)
    start_columns = np.sort(start_columns)
    
    # запускаем maxvol на начальных данных
    return maxvol(matrix, start_rows, start_columns, eps, max_iters)

In [26]:
def F_norm(tensor):
    return np.sum(tensor ** 2)

def truncatedSVD(A, delta):
    u, s, v = np.linalg.svd(A)
    rank = (np.abs(s) >= delta).sum()
    return u[:, :rank], s[:rank], v[:rank, :]

def numel(tensor):
    return tensor.size

In [39]:
def TTCross(tensor, eps):
    d = len(tensor.shape)
    delta = eps / np.sqrt(d - 1) * F_norm(tensor)
    Ctens = tensor.copy()
    
    rk = 1
    r_prev = 1

    G_list = []
    for k in range(1, d):
        Ctens = Ctens.reshape(
            r_prev * tensor.shape[k - 1], 
            numel(Ctens) // (r_prev * tensor.shape[k - 1])
        )
        #u, s, v = truncatedSVD(Ctens, delta)
        u, s, v = Cross(Ctens, delta, 100)
        rk = s.shape[0]
        if k > 1:
            Gk = u.reshape(r_prev, tensor.shape[k - 1], rk)
        else:
            Gk = u.reshape(tensor.shape[k - 1], rk)
        G_list.append(Gk)
        r_prev = rk
        Ctens =  s @ v
    G_list.append(Ctens)
    #G_list.append(Ctens.reshape(r_prev, Ctens.shape[1] // tensor.shape[-1], tensor.shape[-1]))
    return G_list
    

In [40]:
from PIL import Image
import numpy as np

# Загружаем jpg картинку при помощи PIL
image = Image.open('amanita.jpg')

# Переводим картинку в массив numpy
image_array = np.array(image)
image_array = einops.reduce(image_array[1:, 1:, :].astype('float'), "(h 4) (w 4) c -> h w c", 'mean')
"""[1:, 1:]
image_array = (image_array - image_array.min()) / (image_array.max() - image_array.min())
image_array = einops.reduce(image_array, "(h 4) (w 4) c -> h w c", 'mean')"""

# Проверяем размерность массива
print(image_array.shape)

(56, 56, 3)


In [41]:
import matplotlib.pyplot as plt

In [42]:
tensor_train = TTCross(image_array, 1e-8)
rec = einops.einsum(tensor_train[0], tensor_train[1], tensor_train[2], "i alpha, alpha j beta, beta k -> i j k")
plt.imshow((rec - rec.min()) / (rec.max() - rec.min()))

ValueError: cannot reshape array of size 168 into shape (56,1)

In [24]:
tensor_train = TTCross(image_array, 1e-5)
rec = einops.einsum(tensor_train[0], tensor_train[1], tensor_train[2], "i alpha, alpha j beta, beta k -> i j k")
plt.imshow((rec - rec.min()) / (rec.max() - rec.min()))

NameError: name 'search_max_volume_submatrix' is not defined